In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import os
import sys
sys.path.insert(1, '../../framework')
from racetrack import *
rt = RACETrack()

_base_ = '../../../data/kaggle_imdb_600k/international-movies-json/'
_files_ = os.listdir(_base_)
print(f'Loading {len(_files_)} files...')
_jsons_ = []
for i in range(len(_files_)):
    _file_ = _files_[i]
    _txt_  = open(_base_ + _file_).read()
    _json_ = json.loads(_txt_)
    _jsons_.append(_json_)

print('Applying template to json...')

_xform_map_ = '''
__id__              = '$[*]._id'              | MovieID      | uniq
__director__        = '$[*].director.name_id' | DirectorID   | uniq
__director__        --- "directedMovie"  --- __id__ ^^^ "imdb_600k_international_movies"
'''

ofv_dir = rt.ontologyFrameworkInstance(xform_spec=_xform_map_)
ofv_dir.parse(_jsons_)
ofv_dir.df_triples